In [12]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders.wikipedia import WikipediaLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
llm = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.0,
  base_url="https://openai.vocareum.com/v1"
)

In [14]:
loader = WikipediaLoader(
  "Anthony_Hopkins",
  load_max_docs=1,
  doc_content_chars_max=4000
)
docs = loader.load()

In [15]:
len(docs)

1

In [16]:
len(docs[0].page_content)

4000

In [17]:
print(docs[0].page_content)

Sir Philip Anthony Hopkins (born 31 December 1937) is a Welsh actor. Considered one of Britain's most recognisable and prolific actors, he is known for his performances on the screen and stage. Hopkins has received numerous accolades, including two Academy Awards, four BAFTA Awards, two Primetime Emmy Awards, and a Laurence Olivier Award. He has also received the Cecil B. DeMille Award in 2005 and the BAFTA Fellowship for lifetime achievement in 2008. He was knighted by Queen Elizabeth II for his services to drama in 1993.
After graduating from the Royal Welsh College of Music & Drama in 1957, Hopkins trained at RADA (the Royal Academy of Dramatic Art) in London. He was then spotted by Laurence Olivier, who invited him to join the Royal National Theatre in 1965. Productions at the National included King Lear (his favourite Shakespeare play), Coriolanus, Macbeth, and Antony and Cleopatra. In 1985, he received acclaim and a Laurence Olivier Award for his performance in the David Hare pla

Splitter

In [18]:
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=300,
  chunk_overlap=50
)
all_splits = text_splitter.split_documents(docs)

In [19]:
print(f"Split Wikipedia page into {len(all_splits)} sub-documents.")

Split Wikipedia page into 17 sub-documents.


Embeddings

In [20]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", base_url="https://openai.vocareum.com/v1")

Vector Store

In [21]:
vector_store = InMemoryVectorStore(embeddings)

In [22]:
document_ids = vector_store.add_documents(documents=all_splits)
document_ids[:10]

TypeError: 'NoneType' object is not iterable

In [ ]:
retriver = vector_store.as_retriever(search_kwargs={"k": 3})

Prompt

In [ ]:
template = ChatPromptTemplate.from_messages([
  ("system", "You are an assistant for question-answering tasks."),
  ("human", 
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, just say that you don't know. "
    "Use three sentences maximum and keep the answer concise. "
    "\n# Question: \n-> {question} "
    "\n# Context: \n-> {context} "
    "\n# Answer: "),
])

In [ ]:
template.invoke(
  {"context": "##CONTEXT##", "question": "##QUESTION##"}
).to_messages()

In [ ]:
def format_docs(docs):
  formatted = "\n\n-> ".join(doc.page_content for doc in docs)
  return formatted

Generation

In [ ]:
question = "When the SIlence of the Lambs was released?"
context = format_docs(retriver.invoke(question))
messages = template.invoke({'question': question, 'context': context}).to_messages()
answer = llm.invoke(messages)

In [ ]:
print(messages[1].content)

In [ ]:
answer

LCEL

In [ ]:
rag_chain = (
  RunnableParallel(
    context = retriver | format_docs,
    question = RunnablePassthrough()
  )
  | template
  | llm
)

In [ ]:
rag_chain.invoke("When he was born?")